In [ ]:
-- create target tables
CREATE OR REPLACE TABLE raw.contexture_fof.raw_metadata (
    fund STRING,
    document_type STRING,
    document_date STRING,
    source_filename STRING
);

CREATE OR REPLACE TABLE raw.contexture_fof.raw_nvestments (
    company_name STRING,
    inv_type STRING,
    date STRING,
    shares integer,
    cost numeric(21,2),
    fair_value numeric(21,2),
    gain_loss numeric(21,2),
    cost_per_share numeric(21,2),
    fair_value_per_share numeric(21,2)
);

CREATE OR REPLACE TABLE raw.contexture_fof.raw_statement_of_operations (
    type integer,
    total STRING
);

-- Remove for now.
DROP TABLE RAW.CONTEXTURE_FOF.RAW_METADATA;
DROP TABLE RAW.CONTEXTURE_FOF.RAW_NVESTMENTS;
DROP TABLE RAW.CONTEXTURE_FOF.RAW_STATEMENT_OF_OPERATIONS;

In [ ]:
-- create a stage to S3.  No filetype defined for this one.
CREATE OR REPLACE STAGE s3_contexture_bucket
-- URL = 's3://ec-flowpoint-bucket/test_folder/'
URL = 's3://ec-contexture-portal-bucket/'
CREDENTIALS = (AWS_KEY_ID = '--nothing--' AWS_SECRET_KEY = '--nothing--')
DIRECTORY = ( ENABLE = true );
--FILE_FORMAT = (TYPE = 'NONE')
;

In [ ]:
LIST @raw.contexture_fof.s3_contexture_bucket;
-- LIST CC_QUICKSTART_CORTEX_SEARCH_DOCS.DATA.DOCS;

-- SELECT $1 FROM @s3_contexture_bucket;

In [ ]:
import pandas as pd
import boto3
from io import BytesIO
from snowflake.core import Root
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col

# Create Snowpark session
session = get_active_session()

files_in_stage = session.sql("LIST @s3_contexture_bucket").collect()

for file in files_in_stage:
    print(f"file: {file}and key file[file_key]")


In [ ]:
-- USE ROLE ACCOUNTADMIN;
-- CREATE DATABASE 

In [ ]:
--
-- Creates a data chunker so that we can load the contents of files (pdf) into a database so that we can use it for AI purposes.
-- 
create or replace function analytics.functions.text_chunker(pdf_text string)
returns table (chunk varchar)
language python
runtime_version = '3.9'
handler = 'text_chunker'
packages = ('snowflake-snowpark-python', 'langchain')
as
$$
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd

class text_chunker:

    def process(self, pdf_text: str):
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 1512, #Adjust this as you see fit
            chunk_overlap  = 256, #This let's text have some form of overlap. Useful for keeping chunks contextual
            length_function = len
        )
    
        chunks = text_splitter.split_text(pdf_text)
        df = pd.DataFrame(chunks, columns=['chunks'])
        
        yield from df.itertuples(index=False, name=None)
$$;

In [ ]:
-- Big question here ... does this work in DBT?
-- attach this to an insert statement or do it with dbt.
--
select 
    name,
    relative_path, 
    size as file_size,
    file_url, 
    build_scoped_file_url(@raw.contexture_fof.s3_contexture_bucket, relative_path) as scoped_file_url,
    last_modified as file_last_modified,
    md5 as file_md5,
    func.chunk as chunk
from 
    directory(@raw.contexture_fof.s3_contexture_bucket),
    TABLE(
        analytics.functions.text_chunker (
            to_varchar(
                snowflake.cortex.parse_document(
                    @raw.contexture_fof.s3_contexture_bucket, 
                    relative_path, 
                    {'mode': 'LAYOUT'}
                )
            )
        )
    ) as func
 where relative_path ilike '%.pdf'
  -- where relative_path='Cortical Ventures Fund I LP - Espresso Credit US LP - statement - Q3 2024.pdf'
;

In [ ]:
-- Big question here ... does this work in DBT?
-- attach this to an insert statement or do it with dbt.
--
select 
    *
    -- func.chunk as chunk
from 
    directory(@raw.contexture_fof.s3_contexture_bucket),
    -- TABLE(
    --     analytics.functions.text_chunker (
    --         to_varchar(
    --             snowflake.cortex.parse_document(
    --                 @raw.contexture_fof.s3_contexture_bucket, 
    --                 relative_path, 
    --                 {'mode': 'LAYOUT'}
    --             )
    --         )
    --     )
    -- ) as func
 where relative_path ilike '%.pdf'
  -- where relative_path='Cortical Ventures Fund I LP - Espresso Credit US LP - statement - Q3 2024.pdf'
;

In [ ]:
-- alter table staging.dbt_dhagen.int_fund_of_funds__contexture_pdf_chunks rename to staging.dbt_dhagen.int_fund_of_funds__pdf_chunks;
-- select * from staging.dbt_dhagen.int_fund_of_funds__pdf_chunks;

-- create table staging.unstructured.int_fund_of_funds__pdf_chunks as 
-- select * from staging.dbt_dhagen.int_fund_of_funds__pdf_chunks;

drop table STAGING.UNSTRUCTURED.INT_FUND_OF_FUNDS__CONTEXTURE_PDF_CHUNKS;